In [11]:
using CSV
using Dates
using DataFrames
using DataFramesMeta
using PowerSystems
using SiennaPRASInterface
using PRAS

In [12]:
data_dir = "./data"
base_power = 100.0
descriptors = "./user_descriptors.yaml"
generator_mapping_file = "./generator_mapping.yaml"
data = PowerSystemTableData(
    data_dir,
    base_power,
    descriptors;
    # timeseries_metadata_file=timeseries_metadata_file,
    generator_mapping_file=generator_mapping_file,
)
system = System(data) #; time_series_in_memory=true)

┌ Info: Parsing csv data in branch.csv ...
└ @ PowerSystems C:\Users\UEU112259C\.julia\packages\PowerSystems\AHyDB\src\parsers\power_system_table_data.jl:143
┌ Info: Successfully parsed branch.csv
└ @ PowerSystems C:\Users\UEU112259C\.julia\packages\PowerSystems\AHyDB\src\parsers\power_system_table_data.jl:148
┌ Info: Parsing csv data in bus.csv ...
└ @ PowerSystems C:\Users\UEU112259C\.julia\packages\PowerSystems\AHyDB\src\parsers\power_system_table_data.jl:143
┌ Info: Successfully parsed bus.csv
└ @ PowerSystems C:\Users\UEU112259C\.julia\packages\PowerSystems\AHyDB\src\parsers\power_system_table_data.jl:148
┌ Info: Parsing csv data in gen.csv ...
└ @ PowerSystems C:\Users\UEU112259C\.julia\packages\PowerSystems\AHyDB\src\parsers\power_system_table_data.jl:143
┌ Info: Successfully parsed gen.csv
└ @ PowerSystems C:\Users\UEU112259C\.julia\packages\PowerSystems\AHyDB\src\parsers\power_system_table_data.jl:148
┌ Info: Parsing csv data in storage.csv ...
└ @ PowerSystems C:\Users\UEU112

Property,Value
Name,
Description,
System Units Base,DEVICE_BASE
Base Power,100.0
Base Frequency,60.0
Num Components,1194
Type,Count
ACBus,3
Arc,2
Area,3


In [13]:
gen_data = CSV.read("./data/gen.csv", DataFrame)
@rsubset! gen_data :for != 0.0
@select! gen_data :name :for 
@eachrow gen_data begin
    mean_time_to_recovery = 10
    recovery_transition_probability = 1 / mean_time_to_recovery
    outage_transition_probability = (:for * recovery_transition_probability) / (1 - :for)

    c = get_component(ThermalStandard, system, :name)
    transition_data = GeometricDistributionForcedOutage(;
        mean_time_to_recovery=mean_time_to_recovery,  # Units of hours
        outage_transition_probability=outage_transition_probability,  # Probability for outage per hour
    )
    add_supplemental_attribute!(system, c, transition_data)
end
;

In [14]:
clear_time_series!(system)
add_time_series!(system, "./timeseries_pointers_2023.json")
transform_single_time_series!(system, Dates.Hour(8760), Dates.Hour(8760))

┌ Info: Cleared all time series.
└ @ InfrastructureSystems C:\Users\UEU112259C\.julia\packages\InfrastructureSystems\LEg3t\src\hdf5_time_series_storage.jl:734
┌ Warning: There is only one forecast window. Setting interval = empty period
└ @ InfrastructureSystems C:\Users\UEU112259C\.julia\packages\InfrastructureSystems\LEg3t\src\system_data.jl:696
┌ Warning: There is only one forecast window. Setting interval = empty period
└ @ InfrastructureSystems C:\Users\UEU112259C\.julia\packages\InfrastructureSystems\LEg3t\src\system_data.jl:696
┌ Warning: There is only one forecast window. Setting interval = empty period
└ @ InfrastructureSystems C:\Users\UEU112259C\.julia\packages\InfrastructureSystems\LEg3t\src\system_data.jl:696
┌ Warning: There is only one forecast window. Setting interval = empty period
└ @ InfrastructureSystems C:\Users\UEU112259C\.julia\packages\InfrastructureSystems\LEg3t\src\system_data.jl:696
┌ Warning: There is only one forecast window. Setting interval = empty period

In [15]:
# Save SIENNA model to json file
to_json(system, "./models/sienna/miso_base.json", force=true)

┌ Info: Serialized time series data to ./models/sienna\miso_base_time_series_storage.h5.
└ @ InfrastructureSystems C:\Users\UEU112259C\.julia\packages\InfrastructureSystems\LEg3t\src\time_series_storage.jl:104
┌ Info: Serialized System to ./models/sienna/miso_base.json
└ @ PowerSystems C:\Users\UEU112259C\.julia\packages\PowerSystems\AHyDB\src\base.jl:453
┌ Info: Serialized System metadata to ./models/sienna\miso_base_metadata.json
└ @ PowerSystems C:\Users\UEU112259C\.julia\packages\PowerSystems\AHyDB\src\base.jl:478


In [16]:
# Generate PRAS model and save to hdf5(.pras) file
pras_system = generate_pras_system(system, PowerSystems.Area)
savemodel(pras_system, "./models/pras/miso_base.pras")

┌ Info: Unit System changed to UnitSystem.NATURAL_UNITS = 2
└ @ PowerSystems C:\Users\UEU112259C\.julia\packages\PowerSystems\AHyDB\src\base.jl:552
┌ Info: The first timestamp of PRAS System being built is : 2023-01-01T00:00:00+00:00 and last timestamp is : 2023-12-31T23:00:00+00:00 
└ @ SiennaPRASInterface C:\Users\UEU112259C\.julia\packages\SiennaPRASInterface\AECs7\src\PowerSystems2PRAS.jl:830
┌ Info: Processing Area objects in Sienna/Data PowerSystems System... 
└ @ SiennaPRASInterface C:\Users\UEU112259C\.julia\packages\SiennaPRASInterface\AECs7\src\PowerSystems2PRAS.jl:847
┌ Info: The Sienna/Data PowerSystems System has 3 regions based on PSY AggregationTopology : Area.
└ @ SiennaPRASInterface C:\Users\UEU112259C\.julia\packages\SiennaPRASInterface\AECs7\src\PowerSystems2PRAS.jl:850
┌ Info: Processing Generators in PSY System... 
└ @ SiennaPRASInterface C:\Users\UEU112259C\.julia\packages\SiennaPRASInterface\AECs7\src\PowerSystems2PRAS.jl:862
┌ Warning: No GeometricForcedOutage S